# Section 3: Projections, Subspaces, Orthogonality, and QR decomposition

In lecture you have been discussing subspaces and the notion of orthogonality. Generating orthogonal subspaces or an orthonormal basis for matrices can be a powerful numerical tool.

In this section we will explore this idea of orthogonality and how to use it to describe matrices and solve least squares.

## Using QR for least squares

We can use least squares and QR to attempt to classify handwritten digits from the MNIST dataset. This is essentially a single layer perceptron with no activation function. We will use tensorflow to load the data since it has a nice loader to numpy.

In [2]:
import tensorflow as tf
import numpy as np
import scipy
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)
print(y_train.shape)

11490434/11490434 [==============================] - 1s 0us/step
(60000, 28, 28)
(60000,)


We see here that each training example is a 28x28 image. We want each example as a single vector so let's flatten that to shape to make a large data matrix.

In [3]:
x_train = x_train.reshape((60000, 784))
bias = np.ones((60000, 1))
x_train = np.concatenate((bias, x_train), axis=1)

Let's call the data matrix $A$ and the labels $b$. There likely isn't a solution to the system $Ax = b$ since the matrix $A$ has many less columns than rows. This means we want to solve $\min_{x}||Ax - b||_{l_2}$, which is the least squares problem. Let's think about how we can do this using $QR$. 

Fact:
1. Because $Q$ is orthonormal, it doesn't change the norm of any vector.

Proof:
$$||Qy||^2_2 = (Qy)^T (Qy) = y^TQ^TQy = y^ty = ||y||_2^2$$

2. So that means we can transform our minimization problem to:
\begin{align}
&=\min_x ||Ax - b||_2\\
&=\min_x ||Q^T(Ax - b)||_2\\
&=\min_x ||Q^T(QRx - b)||_2\\
&=\min_x ||Rx - Q^T b||_2
\end{align}

But since $R$ is an upper triangular matrix, we know there is a solution to:
$$Rx = Q^T b$$
or
$$x = R^{-1} Q^T b$$
which would make the result of the minimization $0$.

In [4]:
Q, R = scipy.linalg.qr(x_train, mode='economic')
np.linalg.matrix_rank(R)

713

Notice the above technique only works if $R$ is invertible, or full rank. $R$ will only be full rank if the original data, $A$, has linearly dependent columns. Often times there are linear dependencies in the dataset, meaning we have to take a slightly different approach to the $QR$ for least squares.

This process is known as *rank deficient least squares* and requires a modified $QR$ which permutes the $A$ matrix so that the diagonal or $R$ is not increasing. If you're interested, [here is a formal description of this algorithm using householder transformations](https://www.math.usm.edu/lambers/mat610/sum10/lecture11.pdf).

For our case, `scipy` offers a `pivoting` argument flag that does this for us.

In [5]:
Q, R_p, p = scipy.linalg.qr(x_train, mode='economic', pivoting=True)

This version gives us a $Q$ like before but the $R_p$ is provided in the form of:
$$R_p = \begin{bmatrix}
R & S\\
0 & 0
\end{bmatrix} $$
Where the $R$ now is a true upper triangular. The provided $p$ is the pivots required to $A$ to satisfy:
$$A \Pi = Q R_p$$
With $\Pi$ being the permutation matrix created from $p$. We can create that permutation matrix with `np.eye(size)[:,p]`.

We checked previously that the rank of our R matrix is 713 but let's check again with the pivoted R by looking for the first 0 in the diagonal.

In [6]:
rank = np.argmax(np.absolute(np.diag(R_p)) < 1e-6)
rank

713

Now, one way to solve for $x$ is to slice off the part of $R$ that is all zeros and cut the bottom portion off of $Q^T b$ so that we can solve the triangular system. The resulting $x$ needs to be permuted using the pivots to get an actual solution to the least squares.

\begin{align}
||b - Ax||_2^2 &= \left\|b - Q
\begin{bmatrix}
R & S\\
0 & 0
\end{bmatrix}
\Pi^T x \right\|_2^2\\
&= \left \|Q^T b - 
\begin{bmatrix}
R & S\\
0 & 0
\end{bmatrix}
\begin{bmatrix}
u\\
v
\end{bmatrix}
\right\|_2^2\\
&=\left\|
\begin{bmatrix}
c\\
d
\end{bmatrix} -
\begin{bmatrix}
Ru & Sv\\
0
\end{bmatrix} \right\|_2^2\\
&= \|c - Ru - Sv||_2^2 + \|d\|_2^2\\
\text{where} \quad
Q^Tb &=
\begin{bmatrix}
c\\
d
\end{bmatrix}, \quad \Pi^Tx =
\begin{bmatrix}
u\\
v
\end{bmatrix}
\end{align}

In the implementation below, I choose the simplest solution with $v=0$.

In [7]:
R = R_p[:rank, :rank]
c = Q.T[:rank,:] @ y_train
u = scipy.linalg.solve_triangular(R, c, lower=False)
v = np.zeros(785 - rank)
uv = np.concatenate((u,v))
x = np.eye(785)[:,p] @ uv
pred = x_train @ x
print(pred[:10])
print(y_train[:10])

[4.19480069 1.20585109 3.19619527 2.3125187  7.70189204 4.07685415
 1.62278595 3.92588536 1.84626138 4.82906635]
[5 0 4 1 9 2 1 3 1 4]


We see that this is *sort of* working...

The prediction numbers are approximately the same as the labels but with categorical labels like *digit*, getting a real number that is around the correct value isn't going to make a very good classifier.

We can really improve this model by making 10 different binary classifiers by changing the right hand side, $y_{train}$, to be a $1$ or a $0$ depending if it is the digit we are trying to classify.

## Exercise 3

Here we will complete the first step to improve the least squares classifier by implementing it as a binary classifier.

Modify `y_train` so that it *one hot encodes* the dataset. One hot encoding is where you take a vector of categorical labels and translate it to a vector for each category. Each category vector should have a $1$ if the observation is that category and a $0$ otherwise.

### One hot encoding example:
$$
\begin{bmatrix} 3\\ 2\\ 3\\ 1\\ 0 \end{bmatrix} \to
\begin{bmatrix} 0\\ 0\\ 0\\ 0\\ 1 \end{bmatrix}, 
\begin{bmatrix} 0\\ 0\\ 0\\ 1\\ 0 \end{bmatrix}, 
\begin{bmatrix} 0\\ 1\\ 0\\ 0\\ 0 \end{bmatrix}, 
\begin{bmatrix} 1\\ 0\\ 1\\ 0\\ 0 \end{bmatrix}
\quad \text{or in matrix form:} \quad
\begin{bmatrix}
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0
\end{bmatrix}
$$

These new one hot encoded vectors will become the new $b$ in the least squares formulation (formerly `y_train`).

In [8]:
# Your code here
print(y_train)
hot_encoding = np.zeros((len(y_train), 10))
print(hot_encoding)
print(hot_encoding.shape)

for i in range(len(y_train)):
  hot_encoding[i][y_train[i]] = 1

print(hot_encoding)
# Test matrix
print(hot_encoding[0])



[5 0 4 ... 5 6 8]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(60000, 10)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


## Homework 3

After creating the one hot encoding of the labels, use the $Q$ and $R$ from the code above to solve the least squares problem for $x$ for each of these 10 one hot encoded column vectors.

Concat each $x$ into a matrix, $X = \begin{bmatrix} x_1, & x_2, & \dots, & x_{10} \end{bmatrix}$

And get the resulting prediction matrix, $Y = A X$.

For each row (observation) of this prediction matrix, you will have 10 values that correspond to each label. Whichever value is highest is the prediction. Extract the index of the highest value for each row in this matrix. Compare the predicted labels with the actual labels to calculate your prediction accuracy.

### Bonus / Extra Credit

Create a confusion matrix of the result and make a short comment about your observations of this matrix.

In [9]:
# Your code here
X = np.zeros((60000,10))
print(X.shape)
for i in range(0, hot_encoding.shape[1]):
    c = Q.T[:rank,:] @ hot_encoding[:,i]
    u = scipy.linalg.solve_triangular(R, c, lower=False)
    v = np.zeros(785 - rank)
    uv = np.concatenate((u,v))
    x = np.eye(785)[:,p] @ uv
    #instead of next line, do an np thing to add the data
    X[:,i] = x_train @ x
# print(X[:3])
max_indices = np.argmax(X, 1)
max_values = X[np.arange(len(X)),max_indices]
confusion_matrix = np.zeros((10,10))
correct = 0
for i in range(len(y_train)):
    if(y_train[i] == max_indices[i]):
        correct += 1
    confusion_matrix[y_train[i]][max_indices[i]] += 1

correctness = correct/60000
print(correctness * 100, " % accurate")

(60000, 10)
85.77333333333334  % accurate


## A note about least squares in practice

Please don't implement least squares on your own like this in practice, `numpy` has least squares implemented for you.

In [10]:
result = np.linalg.lstsq(x_train, y_train, rcond=1e-6)
pred = x_train @ result[0]

print(pred[:10])
print(y_train[:10])

[4.19480069 1.20585109 3.19619527 2.3125187  7.70189204 4.07685415
 1.62278595 3.92588536 1.84626138 4.82906635]
[5 0 4 1 9 2 1 3 1 4]
